In [ ]:
# Imports and definitions

import numpy as np
import os
from enum import Enum
from convert_board import convert_board

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

NO_MOVE = -1000


In [ ]:
# Convert data to python arrays

boards = []
y = []

def read_line(string):
    values = string.split(",")
    moves_string = values[0]
    evaluations = [int(values[i]) for i in range(1, 8)]
    indexes = [0 for _ in range(7)]
    matrix = np.full((7, 6), BoardFields.EMPTY.value, dtype="int8")
    turn = BoardFields.PLAYER1

    for move_string in moves_string:
        move = int(move_string) - 1
        matrix[move, indexes[move]] = turn.value
        indexes[move] += 1
        turn = (
            BoardFields.PLAYER1 if turn == BoardFields.PLAYER2 else BoardFields.PLAYER2
        )

    for i in range(len(evaluations)):
        if evaluations[i] == NO_MOVE:
            continue
        child_matrix = np.copy(matrix)
        child_matrix[i, indexes[i]] = turn.value
        boards.append(child_matrix)
        y.append(evaluations[i])


for filename in os.listdir(os.getcwd() + "/data"):
    path = os.getcwd() + "/data/" + filename
    try:
        with open(path, "r") as file:
            for line in file:
                read_line(line.strip())
    except FileNotFoundError:
        print("File not found.")
    except IOError:
        print("Error reading the file.")

np.save(os.getcwd() + "/boards.npy", boards)
np.save(os.getcwd() + "/y.npy", y)


In [ ]:
# Convert python arrays into model input

boards = np.load(os.getcwd() + "/boards.npy")
positions = []
my_lines = []
enemy_lines = []

increment = 1
length = len(boards)
count = 0
percentage = 0
for board in boards:
    converted_board = convert_board(board)

    positions.append(converted_board[0])
    my_lines.append(converted_board[1])
    enemy_lines.append(converted_board[2])

    count += 1
    if count > percentage / 100 * length:
        print(percentage, "%")
        percentage += increment

np.save(os.getcwd() + "/positions.npy", positions)
np.save(os.getcwd() + "/my_lines.npy", my_lines)
np.save(os.getcwd() + "/enemy_lines.npy", enemy_lines)


In [ ]:
# Check if data was converted correctly

y = np.load(os.getcwd() + "/y.npy")
boards = np.load(os.getcwd() + "/boards.npy")
my_lines = np.load(os.getcwd() + "/my_lines.npy")
enemy_lines = np.load(os.getcwd() + "/enemy_lines.npy")

print("size:", len(boards))

i = 0
print(boards[i])
print("evaluation:", y[i])
print("player 1 lines:", my_lines[i])
print("player 2 lines:", enemy_lines[i])
